# read me

相比example1做出的修改：


1、解决了gp_x undefined问题 原因是全局变量的作用域只能是一个文件- -，解决措施就是把自定义和函数和gplearn主程序放在同一个文件中。

2、对gplearn进行了修改


   原版：用户指定每一代parent数目；返回公式数目；显示公式数目
    
    
   新版：新增加了一个参数：candidate(candidate数目必须大于tournourment数目,小于parent),即每一代最终会返回candidate个候选者，计算这些候选者的相关性并排列，随机删除前(candidate - parent)组相关性较高因子中的一个，直至只剩余parent个因子,作为下一代的父代。这样做的目的是尽可能地消除共线性。
   
   X_train与y_train的划分方式进行了修改

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import gplearn
import talib
#from gp_extend_func import *
from gplearn import genetic1
from gplearn import genetic
import gplearn
from lib2to3.pgen2.pgen import DFAState

In [2]:
def protected_division(x1, x2):
    with np.errstate(divide='ignore', invalid='ignore'):
        return np.where(np.abs(x2) > 0.001, np.divide(x1, x2), 1.)


def protected_sqrt(x1):
    return np.sqrt(np.abs(x1))


def protected_log(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        return np.where(np.abs(x1) > 0.001, np.log(np.abs(x1)), 0.)


def protected_inverse(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        return np.where(np.abs(x1) > 0.001, 1. / x1, 0.)


def sigmoid(x1):
    with np.errstate(over='ignore', under='ignore'):
        return 1 / (1 + np.exp(-x1))


# 相加
def gp_add(data1, data2):
    if isinstance(data1, float):
        data1 = np.repeat(data1, gp_x.shape[0])
    if isinstance(data2, float):
        data2 = np.repeat(data2, gp_x.shape[0])
    if isinstance(data1, int):
        data1 = np.repeat(data1, gp_x.shape[0])
    if isinstance(data2, int):
        data2 = np.repeat(data2, gp_x.shape[0])
    return np.add(data1, data2)


# 相减
def gp_sub(data1, data2):
    if isinstance(data1, float):
        data1 = np.repeat(data1, gp_x.shape[0])
    if isinstance(data2, float):
        data2 = np.repeat(data2, gp_x.shape[0])
    if isinstance(data1, int):
        data1 = np.repeat(data1, gp_x.shape[0])
    if isinstance(data2, int):
        data2 = np.repeat(data2, gp_x.shape[0])
    return np.subtract(data1, data2)


# 相乘
def gp_mul(data1, data2):
    if isinstance(data1, float):
        data1 = np.repeat(data1, gp_x.shape[0])
    if isinstance(data2, float):
        data2 = np.repeat(data2, gp_x.shape[0])
    if isinstance(data1, int):
        data1 = np.repeat(data1, gp_x.shape[0])
    if isinstance(data2, int):
        data2 = np.repeat(data2, gp_x.shape[0])
    return np.multiply(data1, data2)


# 相除
def gp_div(data1, data2):
    if isinstance(data1, float):
        data1 = np.repeat(data1, gp_x.shape[0])
    if isinstance(data2, float):
        data2 = np.repeat(data2, gp_x.shape[0])
    if isinstance(data1, int):
        data1 = np.repeat(data1, gp_x.shape[0])
    if isinstance(data2, int):
        data2 = np.repeat(data2, gp_x.shape[0])
    return protected_division(data1, data2)


# 两数比大小，取大值
def gp_max(data1, data2):
    if isinstance(data1, float):
        data1 = np.repeat(data1, gp_x.shape[0])
    if isinstance(data2, float):
        data2 = np.repeat(data2, gp_x.shape[0])
    if isinstance(data1, int):
        data1 = np.repeat(data1, gp_x.shape[0])
    if isinstance(data2, int):
        data2 = np.repeat(data2, gp_x.shape[0])
    return np.maximum(data1, data2)


# 两数比大小，取小值
def gp_min(data1, data2):
    if isinstance(data1, float):
        data1 = np.repeat(data1, gp_x.shape[0])
    if isinstance(data2, float):
        data2 = np.repeat(data2, gp_x.shape[0])
    if isinstance(data1, int):
        data1 = np.repeat(data1, gp_x.shape[0])
    if isinstance(data2, int):
        data2 = np.repeat(data2, gp_x.shape[0])
    return np.minimum(data1, data2)


# 绝对值平方根
def gp_sqrt(data):
    if isinstance(data, float):
        return protected_sqrt(np.repeat(data, gp_x.shape[0]))
    if isinstance(data, int):
        return protected_sqrt(np.repeat(data, gp_x.shape[0]))
    return protected_sqrt(data)


# 对数
def gp_log(data):
    if isinstance(data, float):
        return protected_log(np.repeat(data, gp_x.shape[0]))
    if isinstance(data, int):
        return protected_log(np.repeat(data, gp_x.shape[0]))
    return protected_log(data)


# 取相反数
def gp_neg(data):
    if isinstance(data, float):
        return np.negative(np.repeat(data, gp_x.shape[0]))
    if isinstance(data, int):
        return np.negative(np.repeat(data, gp_x.shape[0]))
    return np.negative(data)


# 近零的参数返回0
def gp_inv(data):
    if isinstance(data, float):
        return protected_inverse(np.repeat(data, gp_x.shape[0]))
    if isinstance(data, int):
        return protected_inverse(np.repeat(data, gp_x.shape[0]))
    return protected_inverse(data)


# 取绝对值
def gp_abs(data):
    if isinstance(data, float):
        return np.abs(np.repeat(data, gp_x.shape[0]))
    if isinstance(data, int):
        return np.abs(np.repeat(data, gp_x.shape[0]))
    return np.abs(data)


# 三角函数
def gp_sin(data):
    if isinstance(data, float):
        return np.sin(np.repeat(data, gp_x.shape[0]))
    if isinstance(data, int):
        return np.sin(np.repeat(data, gp_x.shape[0]))
    return np.sin(data)


# 三角函数
def gp_cos(data):
    if isinstance(data, float):
        return np.cos(np.repeat(data, gp_x.shape[0]))
    if isinstance(data, int):
        return np.cos(np.repeat(data, gp_x.shape[0]))
    return np.cos(data)


# 三角函数
def gp_tan(data):
    if isinstance(data, float):
        return np.tan(np.repeat(data, gp_x.shape[0]))
    if isinstance(data, int):
        return np.tan(np.repeat(data, gp_x.shape[0]))
    return np.tan(data)


# 三角函数
def gp_sig(data):
    if isinstance(data, float):
        return sigmoid(np.repeat(data, gp_x.shape[0]))
    if isinstance(data, int):
        return sigmoid(np.repeat(data, gp_x.shape[0]))
    return sigmoid(data)


def rolling_prod(data):
    if isinstance(data, float):
        return np.prod(np.repeat(data, gp_x.shape[0]))
    if isinstance(data, int):
        return np.prod(np.repeat(data, gp_x.shape[0]))
    return np.prod(data)


# 数组10日移动窗口和
def ts_sum(data):
    if isinstance(data, float):
        return np.repeat(data, gp_x.shape[0])
    if isinstance(data, int):
        return np.repeat(data, gp_x.shape[0])
    window = 10
    value = np.array(pd.Series(data.flatten()).rolling(window).sum().tolist())
    value = np.nan_to_num(value)
    return value


# 数组的10日移动平均数
def sma(data):
    if isinstance(data, float):
        return np.repeat(data, gp_x.shape[0])
    if isinstance(data, int):
        return np.repeat(data, gp_x.shape[0])
    window = 10
    value = np.array(pd.Series(data.flatten()).rolling(window).mean().tolist())
    value = np.nan_to_num(value)
    return value


# 数组10日移动窗口标准差
def stddev(data):
    if isinstance(data, float):
        return np.repeat(data, gp_x.shape[0])
    if isinstance(data, int):
        return np.repeat(data, gp_x.shape[0])
    window = 10
    value = np.array(pd.Series(data.flatten()).rolling(window).std().tolist())
    value = np.nan_to_num(value)
    return value


# 数组10日移动窗口元素相乘
def product(data):
    if isinstance(data, float):
        return np.repeat(data, gp_x.shape[0])
    if isinstance(data, int):
        return np.repeat(data, gp_x.shape[0])
    window = 10
    value = np.array(pd.Series(data.flatten()).rolling(10).apply(rolling_prod).tolist())
    value = np.nan_to_num(value)
    return value


# 数组10日移动窗口最小值
def ts_min(data):
    if isinstance(data, float):
        return np.repeat(data, gp_x.shape[0])
    if isinstance(data, int):
        return np.repeat(data, gp_x.shape[0])
    window = 10
    value = np.array(pd.Series(data.flatten()).rolling(window).min().tolist())
    value = np.nan_to_num(value)
    return value


# 数组10日移动窗口最小大值
def ts_max(data):
    if isinstance(data, float):
        return np.repeat(data, gp_x.shape[0])
    if isinstance(data, int):
        return np.repeat(data, gp_x.shape[0])
    window = 10
    value = np.array(pd.Series(data.flatten()).rolling(window).max().tolist())
    value = np.nan_to_num(value)
    return value


# 数组相邻两数差
def delta(data):
    if isinstance(data, float):
        return np.repeat(data, gp_x.shape[0])
    if isinstance(data, int):
        return np.repeat(data, gp_x.shape[0])
    value = np.diff(data.flatten())
    value = np.append(0, value)
    return value


# 往后移一个单位
def delay(data):
    if isinstance(data, float):
        return np.repeat(data, gp_x.shape[0])
    if isinstance(data, int):
        return np.repeat(data, gp_x.shape[0])
    period = 1
    value = pd.Series(data.flatten()).shift(1)
    value = np.nan_to_num(value)
    return value


# 数组10日移动窗口最大值索引
def ts_argmax(data):
    if isinstance(data, float):
        return np.repeat(data, gp_x.shape[0])
    if isinstance(data, int):
        return np.repeat(data, gp_x.shape[0])
    window = 10
    value = pd.Series(data.flatten()).rolling(10).apply(np.argmax) + 1
    value = np.nan_to_num(value)
    return value


# 数组10日移动窗口最小值索引
def ts_argmin(data):
    if isinstance(data, float):
        return np.repeat(data, gp_x.shape[0])
    if isinstance(data, int):
        return np.repeat(data, gp_x.shape[0])
    window = 10
    value = pd.Series(data.flatten()).rolling(10).apply(np.argmin) + 1
    value = np.nan_to_num(value)
    return value

In [3]:
data = pd.read_csv('./data/example.csv')
data['open_time'] = pd.to_datetime(data.open_time)

In [4]:
from Factor import Factor
data = Factor(data)

In [5]:
data.get_AD()
data.get_ADX()
data.get_APO()
data.get_ATR()
data.get_BOP()

In [6]:
df = data.get_df()
df.head(5)

,open_time,open,high,low,close,volume,pct,AD,ADX,APO,ATR,BOP
0,2019-10-01 00:00:00,8284.87,8286.05,8238.01,8242.33,219.186121,-0.005135,-179.765470,30.426678,17.244231,21.767857,-0.885512
1,2019-10-01 00:05:00,8241.55,8249.79,8230.03,8234.22,164.891499,-0.000889,-274.728286,30.426678,17.244231,21.767857,-0.370951
2,2019-10-01 00:10:00,8235.52,8245.07,8206.57,8243.39,175.031226,0.000956,-114.972513,30.426678,17.244231,21.767857,0.204416
3,2019-10-01 00:15:00,8242.94,8252.53,8235.18,8242.69,93.312935,-0.000030,-127.503875,30.426678,17.244231,21.767857,-0.014409
4,2019-10-01 00:20:00,8239.17,8239.31,8210.01,8219.42,163.807780,-0.002397,-186.094508,30.426678,17.244231,21.767857,-0.674061


In [7]:
df['pct'] = df.close.pct_change().fillna(method = 'bfill')

# genetic algorithm

In [8]:
#指定参数与candidate
#进入循环
#gen = 1 former = None else former = former
#代与代之间：1 删除常值 2 按相关性随机剔除
gp_add = gplearn.functions.make_function(function=gp_add, name='gp_add', arity=2)
gp_sub = gplearn.functions.make_function(function=gp_sub, name='gp_sub', arity=2)
gp_mul = gplearn.functions.make_function(function=gp_mul, name='gp_mul', arity=2)
gp_div = gplearn.functions.make_function(function=gp_div, name='gp_div', arity=2)
gp_max = gplearn.functions.make_function(function=gp_max, name='gp_max', arity=2)
gp_min = gplearn.functions.make_function(function=gp_min, name='gp_min', arity=2)
gp_sqrt = gplearn.functions.make_function(function=gp_sqrt, name='gp_sqrt', arity=1)
gp_log = gplearn.functions.make_function(function=gp_log, name='gp_log', arity=1)
gp_neg = gplearn.functions.make_function(function=gp_neg, name='gp_neg', arity=1)
gp_inv = gplearn.functions.make_function(function=gp_inv, name='gp_inv', arity=1)
gp_abs = gplearn.functions.make_function(function=gp_abs, name='gp_abs', arity=1)
gp_sin = gplearn.functions.make_function(function=gp_sin, name='gp_sin', arity=1)
gp_cos = gplearn.functions.make_function(function=gp_cos, name='gp_cos', arity=1)
gp_tan = gplearn.functions.make_function(function=gp_tan, name='gp_tan', arity=1)
gp_sig = gplearn.functions.make_function(function=gp_sig, name='gp_sig', arity=1)
delta = gplearn.functions.make_function(function=delta, name='delta', arity=1)
delay = gplearn.functions.make_function(function=delay, name='delay', arity=1)
sma = gplearn.functions.make_function(function=sma, name='sma', arity=1)
stddev = gplearn.functions.make_function(function=stddev, name='stddev', arity=1)
product = gplearn.functions.make_function(function=product, name='product', arity=1)
ts_min = gplearn.functions.make_function(function=ts_min, name='ts_min', arity=1)
ts_max = gplearn.functions.make_function(function=ts_max, name='ts_max', arity=1)
ts_argmax = gplearn.functions.make_function(function=ts_argmax, name='ts_argmax', arity=1)
ts_argmin = gplearn.functions.make_function(function=ts_argmin, name='ts_argmin', arity=1)
ts_sum = gplearn.functions.make_function(function=ts_sum, name='ts_sum', arity=1)


init_function = [gp_add, gp_sub, gp_mul, gp_div, 
                 gp_sqrt, gp_log, gp_neg, gp_inv, 
                 gp_abs, gp_max, gp_min, gp_sin,
                 gp_cos, gp_tan, gp_sig]
user_function = [delta, delay,sma, stddev, product,ts_min, ts_max, ts_argmax, ts_argmin,ts_sum]

In [9]:
#获得X与y
origin_fator_lst = list(df.columns[7:])
X = df[origin_fator_lst]
y = df['pct'].shift(-1)

In [10]:
#获得X_train 与 y_train;X_test与y_test
train_ratio = 0.8
X_train = X.iloc[:int(X.shape[0]*train_ratio)]
y_train = y.iloc[:int(X.shape[0]*train_ratio)]
X_test = X.iloc[int(X.shape[0]*train_ratio):-1]
y_test = y.iloc[int(X.shape[0]*train_ratio):-1]

In [12]:
def new_factor_choose(gp,candidate):
    #计算新因子
    new_factor_df = pd.DataFrame()
    for g in gp[0]:
        new_factor_df[str(g)]=eval(str(g))
    valid_factor_lst = list(new_factor_df.columns)

    #去除常数
    for factor in new_factor_df.columns:
        if (new_factor_df[factor].std() <1e-8):
            valid_factor_lst.remove(factor)
    corr_df = new_factor_df[valid_factor_lst].corr(method = 'spearman')

    #相关性排序(绝对值)
    re_dic = {}
    for factor1_index in range(len(corr_df.index)):
        for factor2_index in range(factor1_index,len(corr_df.columns)):
            if factor1_index != factor2_index:
                factor1 = corr_df.index[factor1_index]
                factor2 = corr_df.columns[factor2_index]
                key = tuple([factor1,factor2])
                re_dic[key] = abs(corr_df.loc[factor1,factor2])

    sort_lst = sorted(re_dic.keys())

    #按照相关性进行删除
    for i in range(len(sort_lst)):
        tmp = sort_lst[-i]
        rand = np.random.uniform(0,1,1)[0]
        if rand < 1/2:
            drop = tmp[0]
        else:
            drop = tmp[1]
        try:
            valid_factor_lst.remove(drop)
        except:
            pass
        if len(valid_factor_lst) == candidate:
            break
    
    #删除programs
    program = gp[0]
    program_candidate = []
    for factor in valid_factor_lst:
        program_candidate.append(program[list(new_factor_df.columns).index(factor)])
    
    return program_candidate

In [13]:
#gplearn
generations = 3 #迭代的次数
population_size = 20  #种群规模(每一代个体数目即初始树的个数)
tournament_size = 10  #迭代一次进入下一次迭代数量
hall_of_fame = 15 #备选数量
candidate = 15
n_components = 10#返回公式个数
random_state=233 #锁随机子可以随便写个整数
n_jobs = -1 #CPU核心个数.-1为全部核心
function_set = init_function + user_function #迭代的公式
init_method ='half and half'  #‘grow’公式和基础数据随机抽，'full'叶子抽基础数据，节点抽公式，‘half and half’ 前面两种各一半
init_depth =(2,4)  #分枝的的深度

In [14]:
import dill as pickle 

In [ ]:
global gp_x
import gc
gp_x = X_train.values
for factor in origin_fator_lst:
    locals()[factor] = X_train[factor].to_numpy()
for i in range(generations):
    print(f'正在进行第{i+1}代：')
    if i == 0:
        gp0 = genetic1.SymbolicTransformer(
                        feature_names= origin_fator_lst,           #基础行情数据的变量名
                        function_set=function_set,          #迭代的函数公式
                        generations=1,            #迭代的次数
                        metric='spearman',                  #适应度的评价标准
                        population_size=population_size,    #迭代的和序数
                        tournament_size=tournament_size,                
                        random_state=random_state,
                        hall_of_fame = hall_of_fame,                   
                        n_components = 1,
                        n_jobs =n_jobs,
                        init_method=init_method,
                        init_depth = init_depth,
                        verbose = 2,
                        former = None
                         )
        gp0.fit(X_train, y_train)
        former = gp0._programs
        former[0] = new_factor_choose(former,candidate)
        random_state += 1
        del gp0
        gc.collect()
    elif i != (generations-1):
        gp0 = genetic1.SymbolicTransformer(
                        feature_names= origin_fator_lst,           #基础行情数据的变量名
                        function_set=function_set,          #迭代的函数公式
                        generations=1,            #迭代的次数
                        metric='spearman',                  #适应度的评价标准
                        population_size=population_size,    #迭代的和序数
                        tournament_size=tournament_size,                
                        random_state=random_state,
                        hall_of_fame = hall_of_fame,                   
                        n_components = 1,
                        n_jobs =n_jobs,
                        init_method=init_method,
                        init_depth = init_depth,
                        verbose = 2,
                        former = former
                         )
        gp0.fit(X_train, y_train)
        former = gp0._programs
        former[0] = new_factor_choose(former,candidate)
        random_state += 1
        del gp0
        gc.collect()
    else:
        gp0 = genetic1.SymbolicTransformer(
                        feature_names= origin_fator_lst,           #基础行情数据的变量名
                        function_set=function_set,          #迭代的函数公式
                        generations=1,            #迭代的次数
                        metric='spearman',                  #适应度的评价标准
                        population_size=population_size,    #迭代的和序数
                        tournament_size=tournament_size,                
                        random_state=random_state,
                        hall_of_fame = hall_of_fame,                   
                        n_components = n_components,
                        n_jobs =n_jobs,
                        init_method=init_method,
                        init_depth = init_depth,
                        verbose = 2,
                        former = former
                         )
        gp0.fit(X_train, y_train)

正在进行第1代：
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    7.0s remaining:   21.3s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   44.8s finished


   0     5.60        0.0079708        9        0.0449239              N/A      0.00s


In [15]:
import gc
gc.collect()
gp = genetic1.SymbolicTransformer(
                        feature_names= origin_fator_lst,           #基础行情数据的变量名
                        function_set=function_set,          #迭代的函数公式
                        generations=1,            #迭代的次数
                        metric='spearman',                  #适应度的评价标准
                        population_size=population_size,    #迭代的和序数
                        tournament_size=tournament_size,                
                        random_state=random_state,
                        hall_of_fame = 20,               
                        n_components = 5,
                        n_jobs =n_jobs,
                        init_method=init_method,
                        init_depth = init_depth,
                        verbose = 2,
                        former = None
                         )
gp.fit(X_train, y_train)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    9.5s remaining:   28.6s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   24.0s finished


   0     3.90        0.0156786        2        0.0635527              N/A      0.00s


SymbolicTransformer(feature_names=['AD', 'ADOSC', 'ADX', 'ADXR', 'APO',
                                   'aroondown', 'aroonup', 'AROONOSC', 'ATR',
                                   'AVGPRICE', 'BBANDS_upper', 'BBANDS_middle',
                                   'BBANDS_lower', 'BETA', 'BOP', 'CCI', 'CMO',
                                   'CORREL', 'DEMA', 'DX', 'EMA', 'HT_DCPERIOD',
                                   'HT_DCPHASE', 'HT_PHASOR_inphase',
                                   'HT_PHASOR_quadrature', 'HT_SINE_sine',
                                   'HT_SINE_leadsine', 'HT_TRENDMODE', 'KAMA',
                                   'LINEARRE...
                                  <gplearn.functions._Function object at 0x000001D6139A6B80>,
                                  <gplearn.functions._Function object at 0x000001D6139A6670>,
                                  <gplearn.functions._Function object at 0x000001D6139A6C10>,
                                  <gplearn.functions._Function object at 0x000001D6139A68B0>],
                    generations=1, hall_of_fame=20, init_depth=(2, 4),
                    metric='spearman', n_components=5, n_jobs=-1,
                    population_size=30, random_state=233, verbose=2)

In [85]:
#这个放在外面
global gp_x
gp_x = X_train.values
for factor in origin_fator_lst:
    locals()[factor] = X_train[factor].to_numpy()

candidate = 25   
#计算新因子
new_factor_df = pd.DataFrame()
for g in gp._programs[0]:
    new_factor_df[str(g)]=eval(str(g))
valid_factor_lst = list(new_factor_df.columns)

#去除常数
for factor in new_factor_df.columns:
    if (new_factor_df[factor].std() <1e-8):
        valid_factor_lst.remove(factor)
corr_df = new_factor_df[valid_factor_lst].corr(method = 'spearman')

#相关性排序(绝对值)
re_dic = {}
for factor1_index in range(len(corr_df.index)):
    for factor2_index in range(factor1_index,len(corr_df.columns)):
        if factor1_index != factor2_index:
            factor1 = corr_df.index[factor1_index]
            factor2 = corr_df.columns[factor2_index]
            key = tuple([factor1,factor2])
            re_dic[key] = abs(corr_df.loc[factor1,factor2])
            
sort_lst = sorted(re_dic.keys())

#按照相关性进行删除
for i in range(len(sort_lst)):
    tmp = sort_lst[-i]
    rand = np.random.uniform(0,1,1)[0]
    if rand < 1/2:
        drop = tmp[0]
    else:
        drop = tmp[1]
    try:
        valid_factor_lst.remove(drop)
    except:
        pass
    if len(valid_factor_lst) == candidate:
        break
len(valid_factor_lst)

25

In [82]:
valid_factor_lst

25

In [92]:
program = gp._programs[0]
program_candidate = []
for factor in valid_factor_lst:
    program_candidate.append(program[list(new_factor_df.columns).index(factor)])


In [94]:
len(program_candidate)

25